In [34]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from sklearn.metrics import cohen_kappa_score

def get_score(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def to_bins(x, borders):
    for i in range(len(borders)):
        if x <= borders[i]:
            return i
    return len(borders)

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _loss(self, coef, X, y, idx):
        X_p = np.array([to_bins(pred, coef) for pred in X])
        ll = -get_score(y, X_p)
        return ll

    def fit(self, X, y):
        coef = [0.2, 0.4, 0.6, 0.8]
        golden1 = 0.618
        golden2 = 1 - golden1
        ab_start = [(0.01, 0.3), (0.15, 0.56), (0.35, 0.75), (0.6, 0.9)]
        for it1 in range(10):
            for idx in range(4):
                # golden section search
                a, b = ab_start[idx]
                # calc losses
                coef[idx] = a
                la = self._loss(coef, X, y, idx)
                coef[idx] = b
                lb = self._loss(coef, X, y, idx)
                for it in range(20):
                    # choose value
                    if la > lb:
                        a = b - (b - a) * golden1
                        coef[idx] = a
                        la = self._loss(coef, X, y, idx)
                    else:
                        b = b - (b - a) * golden2
                        coef[idx] = b
                        lb = self._loss(coef, X, y, idx)
        self.coef_ = {'x': coef}

    def predict(self, X, coef):
        X_p = np.array([to_bins(pred, coef) for pred in X])
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [27]:
pred = []
test = []
for seed in [0, 10, 50, 88, 150, 255, 777, 1022, 2019, 2100]:
    for fold_id in range(5):
        if fold_id==0:
            y_pred = np.load("th/pred_seed{}_fold{}.npy".format(seed, fold_id))
            y_true = np.load("th/true_seed{}_fold{}.npy".format(seed, fold_id))
            y_pred = rankdata(y_pred)/len(y_pred)
        else:
            y_pred_ = np.load("th/pred_seed{}_fold{}.npy".format(seed, fold_id))
            y_pred_ = rankdata(y_pred_)/len(y_pred_)
            y_pred = np.append(y_pred, y_pred_)
            y_true = np.append(y_true, np.load("th/true_seed{}_fold{}.npy".format(seed, fold_id)))
    pred.append(y_pred)
    test.append(y_true)

In [28]:
predict = np.mean(np.array(pred), axis=0)
true = test[0]

In [35]:
optR = OptimizedRounder()
optR.fit(predict, true)
coefficients = optR.coefficients()
y_pred_opt = optR.predict(predict, coefficients)
ensemble_score = get_score(true, y_pred_opt)
ensemble_score

0.451553951784728

In [36]:
coefficients

[0.02853523096120352,
 0.21838191967341264,
 0.5115074678120327,
 0.7155101165408786]

In [38]:
scores = []
for seed in [0, 10, 50, 88, 150, 255, 777, 1022, 2019, 2100]:
    score = []
    for fold_id in range(5):
        y_pred = np.load("th/pred_seed{}_fold{}.npy".format(seed, fold_id))
        y_true = np.load("th/true_seed{}_fold{}.npy".format(seed, fold_id))
        y_pred = rankdata(y_pred)/len(y_pred)
        y_pred_opt = optR.predict(y_pred, coefficients)
        score.append(get_score(y_true, y_pred_opt))
    scores.extend(score)
    
    score_cols = ["score{}".format(i) for i in range(len(scores))]
    coef_cols = ["coef{}".format(i) for i in range(len(coefficients))]
    df = pd.DataFrame([scores + coefficients + [ensemble_score]], columns=score_cols+coef_cols+["ensemble_score"])

In [39]:
np.mean(scores), np.min(scores), np.max(scores), ensemble_score

(0.4448303119111512,
 0.4202198099620118,
 0.46526161883619954,
 0.451553951784728)

In [37]:
p_list = []
t_list = []
for seed in [0, 10, 50, 88, 150, 255, 777, 1022, 2019, 2100]:
    for fold_id in range(5):
        y_pred = np.load("th/pred_seed{}_fold{}.npy".format(seed, fold_id))
        y_pred = rankdata(y_pred)/len(y_pred)
        y_true = np.load("th/true_seed{}_fold{}.npy".format(seed, fold_id))
        p_list.append(y_pred)
        t_list.append(y_true)

In [40]:
import GPy
import GPyOpt

a = []
score_cols = ["score{}".format(i) for i in range(50)]
coef_cols = ["coef{}".format(i) for i in range(4)]
cols = score_cols+coef_cols+["ensemble_score"]

def training(x):
    coefficients = [float(x[:, 0]), float(x[:, 1]), float(x[:, 2]), float(x[:, 3])]
    scores = []
    extend = scores.extend
    for seed in range(10):
        score = []
        append = score.append
        for fold in range(5):
            y_pred_opt = optR.predict(p_list[fold+seed*5], coefficients)
            append(get_score(t_list[fold+seed*5], y_pred_opt))
        extend(score)

    y_pred_opt = optR.predict(predict, coefficients)
    ensemble_score = get_score(true, y_pred_opt)

    #df_ = pd.DataFrame([scores + coefficients + [ensemble_score]], columns=cols)
    a.append([scores + coefficients + [ensemble_score]])
    print(round(np.mean(scores), 5), round(np.min(scores), 5), round(np.max(scores), 5), 
          round(ensemble_score, 5), [round(c, 5) for c in coefficients])

    return -1 * (np.min(scores)+ensemble_score)

bounds = [{'name': 'c1', 'type': 'continuous', 'domain': (0.005, 0.2)},
          {'name': 'c2', 'type': 'continuous', 'domain': (0.15, 0.56)},
          {'name': 'c3', 'type': 'continuous', 'domain': (0.35, 0.75)},
          {'name': 'c4', 'type': 'continuous', 'domain': (0.6, 0.9)},
          ]
myBopt = GPyOpt.methods.BayesianOptimization(f=training, domain=bounds, initial_design_numdata=5, acquisition_type='EI',
                                            num_cores=10)
myBopt.run_optimization(max_iter=2000)

0.42639 0.40257 0.44772 0.43156 [0.13641, 0.32948, 0.46694, 0.76285]
0.4339 0.40434 0.45429 0.43777 [0.10732, 0.25803, 0.46326, 0.70602]
0.4139 0.38924 0.43769 0.41725 [0.02678, 0.34092, 0.72873, 0.77525]
0.43022 0.40447 0.45418 0.43404 [0.05934, 0.30137, 0.3942, 0.74812]
0.41459 0.39149 0.44207 0.41809 [0.05927, 0.27095, 0.70943, 0.84785]
0.40797 0.37974 0.43122 0.41018 [0.14514, 0.17677, 0.37818, 0.62661]
0.43826 0.41307 0.45853 0.44334 [0.07256, 0.28306, 0.46329, 0.7324]
0.44353 0.41737 0.46879 0.44906 [0.03477, 0.28135, 0.48958, 0.73298]
0.43732 0.40625 0.46926 0.44288 [0.005, 0.32183, 0.50054, 0.68108]
0.44216 0.41832 0.46466 0.44606 [0.005, 0.23633, 0.49304, 0.77531]
0.44119 0.41381 0.46611 0.44807 [0.01148, 0.29163, 0.49578, 0.78026]
0.4459 0.41998 0.46877 0.44824 [0.03514, 0.2492, 0.51587, 0.74248]
0.37937 0.35956 0.40443 0.38025 [0.005, 0.56, 0.35, 0.9]
0.40171 0.37868 0.42092 0.40183 [0.005, 0.15, 0.75, 0.6]
0.36762 0.34853 0.38634 0.36974 [0.2, 0.56, 0.75, 0.6]
0.44324 0.419

KeyboardInterrupt: 

In [41]:
myBopt = GPyOpt.methods.BayesianOptimization(f=training, domain=bounds, acquisition_type='EI',
                                            num_cores=10)
myBopt.run_optimization(max_iter=300)

0.42228 0.39781 0.44125 0.42552 [0.12377, 0.24384, 0.38745, 0.713]
0.4272 0.4047 0.44357 0.43333 [0.079, 0.3015, 0.45758, 0.85456]
0.37282 0.35849 0.3962 0.37407 [0.19999, 0.39027, 0.74693, 0.89207]
0.44377 0.41851 0.4674 0.44483 [0.05009, 0.20618, 0.58946, 0.7378]
0.39809 0.37629 0.41532 0.40338 [0.1805, 0.45125, 0.42809, 0.84993]
0.44008 0.4154 0.45756 0.44358 [0.005, 0.17045, 0.52277, 0.79008]
0.42225 0.3888 0.44839 0.42562 [0.005, 0.15, 0.62078, 0.60911]
0.43396 0.4121 0.45234 0.43563 [0.099, 0.15275, 0.56202, 0.8006]
0.44386 0.41961 0.46694 0.44537 [0.005, 0.25659, 0.55905, 0.7743]
0.43359 0.40728 0.45949 0.43404 [0.005, 0.19435, 0.63189, 0.80382]
0.44642 0.42255 0.46883 0.4494 [0.03107, 0.23773, 0.53075, 0.73826]
0.39188 0.3605 0.41757 0.39502 [0.005, 0.56, 0.35, 0.6]
0.37271 0.35115 0.39653 0.37363 [0.005, 0.56, 0.75, 0.6]
0.38635 0.36815 0.40149 0.38498 [0.005, 0.15, 0.35, 0.9]
0.44453 0.42236 0.463 0.44735 [0.06158, 0.23419, 0.52954, 0.76107]
0.40997 0.3872 0.43437 0.41475 [0.

0.44664 0.42383 0.46751 0.44877 [0.03326, 0.2277, 0.53819, 0.75304]
0.43016 0.40533 0.44761 0.43505 [0.14692, 0.32357, 0.54667, 0.7484]
0.44664 0.42383 0.46751 0.44888 [0.03317, 0.22768, 0.5383, 0.75305]
0.44664 0.42383 0.46751 0.44888 [0.03315, 0.22768, 0.53827, 0.75301]
0.38972 0.36734 0.41066 0.39105 [0.18552, 0.41877, 0.74915, 0.60836]
0.39571 0.36864 0.41792 0.39816 [0.17861, 0.17979, 0.35098, 0.60316]
0.4467 0.42383 0.4674 0.44877 [0.03333, 0.22755, 0.5384, 0.75296]
0.42248 0.39011 0.45134 0.4245 [0.05262, 0.39912, 0.36362, 0.75659]
0.44668 0.42386 0.46741 0.44868 [0.03343, 0.22767, 0.53871, 0.75307]
0.44125 0.42121 0.45735 0.44597 [0.06278, 0.21955, 0.49983, 0.80329]
0.44636 0.42251 0.46864 0.44819 [0.03368, 0.22547, 0.53356, 0.75803]
0.44199 0.41358 0.46872 0.44644 [0.02241, 0.27693, 0.60158, 0.73433]
0.4469 0.42271 0.46726 0.44937 [0.03116, 0.22904, 0.54197, 0.7505]
0.44679 0.42429 0.46689 0.44926 [0.03228, 0.22816, 0.54004, 0.75235]
0.44677 0.42407 0.46741 0.44934 [0.03212, 0

0.44663 0.42501 0.46742 0.44858 [0.04143, 0.22153, 0.53921, 0.74386]
0.44298 0.41854 0.46232 0.44797 [0.0718, 0.21113, 0.5287, 0.72859]
0.39848 0.37442 0.41628 0.39887 [0.18435, 0.25111, 0.74612, 0.74006]
0.44621 0.42175 0.46409 0.44921 [0.03193, 0.1921, 0.545, 0.73668]
0.44054 0.40974 0.46448 0.44545 [0.02188, 0.34106, 0.54224, 0.71404]
0.44708 0.42341 0.46391 0.44915 [0.03971, 0.21813, 0.5443, 0.74741]
0.44685 0.42532 0.46775 0.44861 [0.03981, 0.22109, 0.53891, 0.74364]
0.42142 0.39434 0.45114 0.42649 [0.10319, 0.37483, 0.50934, 0.61372]
0.41251 0.3842 0.4328 0.41759 [0.00666, 0.49586, 0.61035, 0.71405]
0.4004 0.38039 0.42382 0.40267 [0.06711, 0.43256, 0.74772, 0.73435]
0.44605 0.41915 0.46494 0.44928 [0.03562, 0.21505, 0.54489, 0.70846]
0.44315 0.41601 0.46315 0.44616 [0.03996, 0.18755, 0.5612, 0.68943]
0.44429 0.41362 0.46486 0.44673 [0.01676, 0.212, 0.57555, 0.75197]
0.40807 0.37756 0.43136 0.40901 [0.01029, 0.53753, 0.3546, 0.74653]
0.40981 0.39044 0.42447 0.41336 [0.19211, 0.151

In [48]:
score_cols = ["score{}".format(i) for i in range(50)]
coef_cols = ["coef{}".format(i) for i in range(4)]
cols = score_cols+coef_cols+["ensemble_score"]
df_ = pd.DataFrame(np.array(a).reshape(-1, 55), columns=cols)
df_

,score0,score1,score2,score3,score4,score5,score6,score7,score8,score9,...,score45,score46,score47,score48,score49,coef0,coef1,coef2,coef3,ensemble_score
0,0.415293,0.440385,0.408084,0.434388,0.436312,0.417129,0.442368,0.413803,0.433583,0.432181,...,0.417333,0.437212,0.409105,0.434388,0.432181,0.136407,0.329482,0.466945,0.762847,0.431558
1,0.427498,0.442790,0.412233,0.434535,0.446750,0.421512,0.446134,0.416715,0.436420,0.445716,...,0.427926,0.447806,0.410313,0.443330,0.444475,0.107317,0.258033,0.463264,0.706022,0.437766
2,0.412689,0.404037,0.398004,0.427426,0.417251,0.407980,0.411726,0.401082,0.428592,0.419941,...,0.407274,0.413082,0.397294,0.429992,0.421062,0.026777,0.340916,0.728730,0.775251,0.417251
3,0.423718,0.438602,0.408099,0.432634,0.437940,0.420758,0.445947,0.414230,0.438639,0.439480,...,0.422807,0.449953,0.414230,0.438417,0.438820,0.059345,0.301367,0.394198,0.748122,0.434044
4,0.414266,0.401460,0.391489,0.428251,0.417951,0.414017,0.403369,0.397256,0.431953,0.418424,...,0.411526,0.404561,0.399763,0.432177,0.424098,0.059267,0.270946,0.709426,0.847847,0.418093
5,0.398342,0.426382,0.393554,0.406233,0.425800,0.404062,0.425575,0.385427,0.403846,0.413641,...,0.399976,0.423357,0.391725,0.410808,0.422412,0.145139,0.176769,0.378176,0.626607,0.410176
6,0.433074,0.445852,0.415532,0.444791,0.446645,0.429044,0.452626,0.422463,0.446107,0.447942,...,0.431283,0.452189,0.414414,0.448080,0.446212,0.072564,0.283064,0.463293,0.732403,0.443342
7,0.438084,0.451106,0.424916,0.450128,0.452164,0.428161,0.456866,0.431283,0.452669,0.453531,...,0.437139,0.461013,0.419020,0.448281,0.446926,0.034767,0.281347,0.489577,0.732979,0.449057
8,0.425170,0.457527,0.415700,0.442396,0.453695,0.418241,0.466779,0.419617,0.445555,0.447002,...,0.426093,0.460686,0.410862,0.441944,0.449010,0.005000,0.321833,0.500537,0.681081,0.442878
9,0.439710,0.452565,0.428315,0.445946,0.451976,0.424934,0.459503,0.431736,0.447231,0.452990,...,0.433905,0.463614,0.423842,0.452112,0.457048,0.005000,0.236326,0.493037,0.775309,0.446058


In [61]:
df_.to_csv("opt_threshold.csv", index=False)

In [58]:
dfs = df_.copy()
dfs["min"] = df_[score_cols].min(axis=1)
dfs["max"] = df_[score_cols].max(axis=1)
dfs["median"] = df_[score_cols].median(axis=1)
dfs["mean"] = df_[score_cols].mean(axis=1)
dfs["std"] = df_[score_cols].std(axis=1)
dfs["ensemble_score_rank"] = df_["ensemble_score"].rank(ascending=False)
dfs["min_score_rank"] = dfs["min"].rank(ascending=False)
dfs["mean_score_rank"] = dfs["mean"].rank(ascending=False)
dfs.sort_values(by="min", ascending=False).drop(score_cols, axis=1)

,coef0,coef1,coef2,coef3,ensemble_score,min,max,median,mean,std,ensemble_score_rank,min_score_rank,mean_score_rank
306,0.039800,0.224172,0.538354,0.745387,0.448431,0.425604,0.467899,0.449073,0.446661,0.011924,212.0,1.0,103.0
307,0.040377,0.223784,0.538158,0.745055,0.448742,0.425558,0.468520,0.448736,0.446651,0.011927,147.0,2.0,108.0
305,0.039259,0.224526,0.538489,0.745712,0.448482,0.425441,0.468109,0.449407,0.446791,0.012003,200.0,3.0,39.0
369,0.039809,0.221088,0.538910,0.743640,0.448614,0.425322,0.467750,0.448925,0.446855,0.011765,179.0,4.0,14.0
310,0.041064,0.223170,0.537707,0.744723,0.448381,0.425250,0.468883,0.448711,0.446606,0.012016,222.0,5.0,123.0
301,0.038655,0.224884,0.538969,0.746001,0.448694,0.425205,0.467857,0.449409,0.446759,0.011992,161.0,6.0,55.0
405,0.040357,0.221190,0.539390,0.742942,0.448725,0.425060,0.466727,0.449576,0.446815,0.011774,150.0,7.0,29.0
327,0.041241,0.223074,0.538961,0.744484,0.448774,0.425013,0.468440,0.448892,0.446574,0.011994,134.0,8.0,131.0
363,0.041435,0.221532,0.539207,0.743864,0.448577,0.425010,0.467425,0.448522,0.446628,0.011880,187.0,9.0,117.0
341,0.041711,0.223136,0.538980,0.745695,0.448621,0.424995,0.467301,0.448424,0.446534,0.012015,177.0,10.0,135.0


In [60]:
dfs.sort_values(by="ensemble_score", ascending=False).drop(score_cols, axis=1)

,coef0,coef1,coef2,coef3,ensemble_score,min,max,median,mean,std,ensemble_score_rank,min_score_rank,mean_score_rank
64,0.027238,0.275151,0.517183,0.748005,0.450401,0.419763,0.467049,0.449552,0.445143,0.012667,1.0,236.0,199.0
65,0.027524,0.276208,0.517350,0.747819,0.450375,0.420233,0.468540,0.449560,0.445173,0.012746,2.0,210.0,196.0
66,0.028899,0.280570,0.518463,0.747026,0.450297,0.420018,0.468505,0.449549,0.445180,0.012764,3.0,224.0,195.0
67,0.030103,0.283956,0.519537,0.746472,0.450237,0.418438,0.469345,0.449525,0.444994,0.013051,4.0,251.0,230.0
78,0.027035,0.274481,0.517456,0.748187,0.450077,0.419731,0.467142,0.449718,0.445181,0.012654,5.0,237.0,194.0
77,0.027080,0.274530,0.517506,0.748170,0.450057,0.419849,0.467158,0.449732,0.445212,0.012637,6.0,233.0,191.0
63,0.027084,0.274505,0.517108,0.748116,0.449980,0.420097,0.466784,0.449628,0.445136,0.012591,7.0,217.0,201.0
79,0.026989,0.274331,0.517398,0.748221,0.449954,0.419315,0.467703,0.449575,0.445150,0.012692,8.0,242.0,198.0
82,0.026814,0.274302,0.517460,0.748324,0.449904,0.419777,0.467703,0.449575,0.445113,0.012680,9.0,234.5,205.5
85,0.026727,0.273977,0.517327,0.748387,0.449898,0.420246,0.467329,0.449445,0.445127,0.012572,10.0,208.5,203.0


In [7]:
from itertools import product
from tqdm import tqdm

c1 = np.linspace(1.5, 1.7, 50)
c2 = np.linspace(2.0, 2.2, 50)
c3 = np.linspace(2.4, 2.6, 50)
c4 = np.linspace(2.8, 3.0, 50)

score_cols = ["score{}".format(i) for i in range(50)]
coef_cols = ["coef{}".format(i) for i in range(4)]
cols = score_cols+coef_cols+["ensemble_score"]

for coefficients in tqdm(product(c1, c2, c3, c4)):
    coefficients = list(coefficients)
    scores = []
    extend = score
    for seed in range(10):
        score = []
        append = score.append
        for fold in range(5):
            y_pred_opt = optR.predict(p_list[fold+seed*5], coefficients)
            append(get_score(t_list[fold+seed*5], y_pred_opt))
        scores.extend(score)
        
    y_pred_opt = optR.predict(predict, coefficients)
    ensemble_score = get_score(true, y_pred_opt)

    df_ = pd.DataFrame([scores + coefficients + [ensemble_score]], columns=cols)
    df = df.append(df_)

1419it [04:32,  5.23it/s]

KeyboardInterrupt: 

In [ ]:
0.44025 0.40983 0.46942 0.44323 [1.70452, 2.12577, 2.60163, 3.00153]
0.43937 0.41626 0.45859 0.443 [1.58434, 2.08215, 2.39131, 2.89942]
0.43289 0.40201 0.4602 0.43459 [1.4267, 2.14224, 2.67265, 3.00413]
0.43051 0.39897 0.45547 0.43358 [1.61883, 2.09116, 2.6941, 2.70234]
0.42651 0.40086 0.44435 0.43067 [1.71761, 2.19846, 2.32668, 3.06271]
0.44 0.41672 0.45899 0.44323 [1.57799, 2.07282, 2.39878, 2.90059]
0.43797 0.41465 0.45628 0.44088 [1.56228, 2.02581, 2.37489, 2.85582]
0.44436 0.42278 0.46151 0.44875 [1.62006, 2.04148, 2.45858, 2.91974]
0.44264 0.41934 0.46091 0.44803 [1.65757, 1.99125, 2.47632, 2.93529]
0.44544 0.42013 0.4621 0.4493 [1.65283, 2.05599, 2.48434, 2.88386]
0.44494 0.42104 0.46237 0.4502 [1.60575, 2.04079, 2.50594, 2.93307]
0.41536 0.39259 0.44044 0.41874 [1.8, 1.9, 2.3, 2.7]
0.42718 0.4033 0.45218 0.4319 [1.50723, 1.9, 2.48358, 3.1]
0.4448 0.41887 0.4609 0.44854 [1.64012, 2.06029, 2.48487, 2.93952]
0.44447 0.42015 0.46474 0.44778 [1.6037, 2.01461, 2.49242, 2.89388]
0.41565 0.38757 0.4536 0.4184 [1.4, 2.3, 2.3, 2.7]
0.42168 0.39389 0.44871 0.42514 [1.8, 1.9, 2.7, 2.99622]
0.44515 0.42293 0.462 0.44933 [1.62088, 2.0398, 2.48701, 2.91566]
0.42429 0.39318 0.45305 0.42799 [1.8, 2.3, 2.7, 2.82974]
0.4454 0.42237 0.46456 0.44965 [1.62001, 2.04181, 2.49209, 2.91501]
0.4182 0.38716 0.44042 0.42127 [1.64091, 2.3, 2.7, 3.1]
0.41614 0.3894 0.44801 0.41943 [1.8, 2.3, 2.3, 2.7]
0.40983 0.38967 0.43143 0.4134 [1.8, 1.9, 2.3, 3.1]
0.41065 0.3813 0.4369 0.41346 [1.4, 2.3, 2.3, 3.1]
0.42396 0.39447 0.44926 0.42709 [1.4, 1.9, 2.7, 2.93609]
0.41956 0.38547 0.44528 0.42277 [1.4, 2.3, 2.7, 2.7]
0.43954 0.41032 0.46254 0.44357 [1.60784, 2.02958, 2.61189, 2.99626]
0.44575 0.42342 0.46421 0.44974 [1.62326, 2.04721, 2.49649, 2.91286]
0.44568 0.42223 0.46381 0.44924 [1.6226, 2.05111, 2.50077, 2.9102]
0.4458 0.42237 0.46401 0.44929 [1.62236, 2.05051, 2.50028, 2.91058]
0.44583 0.42232 0.46399 0.44931 [1.622, 2.05035, 2.50026, 2.91068]
0.44578 0.42232 0.46399 0.4493 [1.62168, 2.05003, 2.50008, 2.91088]
0.44581 0.42326 0.46373 0.44927 [1.62139, 2.0496, 2.49976, 2.91115]
0.41469 0.3908 0.4463 0.41741 [1.4, 1.9, 2.3, 2.7]
0.44329 0.41515 0.46544 0.44639 [1.68096, 2.06212, 2.59316, 2.88807]
0.44571 0.42259 0.46371 0.44939 [1.61752, 2.05106, 2.49819, 2.91024]
0.40829 0.38255 0.43207 0.4111 [1.4, 1.9, 2.3, 3.1]
0.41962 0.38905 0.44611 0.42344 [1.8, 1.9, 2.7, 2.7]
0.44396 0.41568 0.46675 0.44843 [1.59731, 2.1259, 2.56562, 2.89012]
0.44574 0.42288 0.46355 0.44901 [1.61726, 2.0523, 2.4977, 2.90602]
0.44574 0.42288 0.46376 0.44899 [1.61674, 2.05222, 2.49772, 2.90612]
0.44571 0.42244 0.46365 0.44903 [1.6163, 2.05213, 2.49772, 2.90631]
0.4397 0.41397 0.45563 0.44236 [1.65463, 2.01168, 2.3993, 2.89377]
0.44553 0.42149 0.46344 0.4489 [1.60557, 2.05401, 2.50049, 2.90768]
0.44445 0.41679 0.46735 0.45046 [1.64198, 2.15696, 2.50581, 2.79359]
0.43771 0.41353 0.46648 0.44367 [1.60958, 2.11593, 2.44875, 2.7]
0.44596 0.42117 0.46377 0.45021 [1.61507, 2.06836, 2.49838, 2.8842]
0.44574 0.42102 0.46356 0.44992 [1.61568, 2.06659, 2.49839, 2.88844]
0.44483 0.42173 0.46706 0.45079 [1.60753, 2.11541, 2.46416, 2.81897]
0.44555 0.42119 0.46685 0.45156 [1.61583, 2.10419, 2.49135, 2.81528]
0.4447 0.41814 0.46743 0.44969 [1.58447, 2.12821, 2.49277, 2.80334]
0.44501 0.42188 0.46575 0.45158 [1.62601, 2.10318, 2.47341, 2.82306]
0.42415 0.39759 0.44228 0.42948 [1.8, 2.3, 2.46695, 3.1]
0.44458 0.41997 0.46527 0.45011 [1.62455, 2.09442, 2.45923, 2.80708]
0.44545 0.4205 0.4679 0.45048 [1.62668, 2.11603, 2.48761, 2.83933]
0.44606 0.42236 0.46586 0.45064 [1.62121, 2.08063, 2.49189, 2.85598]
0.44606 0.42236 0.46586 0.45058 [1.62116, 2.08062, 2.49182, 2.85499]
0.44606 0.42226 0.4657 0.45062 [1.62105, 2.08086, 2.49164, 2.85427]
0.44609 0.42202 0.46569 0.45056 [1.62093, 2.08124, 2.49139, 2.85353]
0.44601 0.42155 0.46613 0.45055 [1.62077, 2.08194, 2.49099, 2.8529]
0.44593 0.42175 0.46613 0.45035 [1.62062, 2.08293, 2.49046, 2.85254]
0.44594 0.42217 0.46531 0.45063 [1.62047, 2.08403, 2.48988, 2.85173]
0.44519 0.42205 0.46418 0.45083 [1.61622, 2.09563, 2.48013, 2.8325]
0.43584 0.40467 0.45472 0.44033 [1.67168, 1.99823, 2.52084, 3.1]
0.44534 0.42145 0.46457 0.45085 [1.6171, 2.09605, 2.48143, 2.83198]
0.43498 0.4022 0.45731 0.43918 [1.8, 2.14334, 2.5623, 2.7]
0.44532 0.42271 0.46506 0.45078 [1.61984, 2.09673, 2.48537, 2.83142]
0.4455 0.42293 0.46676 0.45082 [1.62148, 2.09685, 2.48861, 2.82958]
0.44235 0.4204 0.46233 0.44845 [1.60968, 2.00388, 2.47504, 2.95801]
0.42379 0.39522 0.45235 0.42869 [1.8, 2.11071, 2.7, 3.1]
0.44553 0.42322 0.4676 0.45079 [1.62216, 2.09651, 2.4907, 2.82839]
0.44603 0.422 0.46854 0.45104 [1.62688, 2.09581, 2.49914, 2.82474]
0.41397 0.3849 0.44421 0.4173 [1.4, 1.9, 2.7, 3.1]
0.44558 0.42285 0.46834 0.45075 [1.62353, 2.09625, 2.49319, 2.82683]
0.44582 0.42215 0.46826 0.451 [1.62444, 2.09587, 2.4947, 2.82641]
0.41941 0.38557 0.44511 0.42268 [1.4, 1.9, 2.7, 2.7]
0.44561 0.42285 0.46834 0.45075 [1.62368, 2.09627, 2.49324, 2.82685]
0.43079 0.40134 0.4604 0.43349 [1.65513, 2.3, 2.5382, 2.7]
0.4152 0.38374 0.43782 0.41844 [1.4, 2.3, 2.7, 3.1]
0.43458 0.40163 0.46092 0.43714 [1.60533, 2.14117, 2.7, 2.90362]
0.42757 0.4 0.45649 0.43133 [1.58799, 2.13223, 2.3, 2.73174]
0.4458 0.42189 0.46853 0.45013 [1.62441, 2.08782, 2.49275, 2.83301]
0.44577 0.422 0.46819 0.44999 [1.6237, 2.08849, 2.49203, 2.83497]
0.44569 0.42185 0.46819 0.45002 [1.62325, 2.08903, 2.49153, 2.83616]
0.4458 0.4218 0.46891 0.44976 [1.62286, 2.08935, 2.49117, 2.83712]
0.44572 0.42146 0.46891 0.44952 [1.62235, 2.08925, 2.49095, 2.83837]
0.44574 0.42052 0.46843 0.4496 [1.62149, 2.08832, 2.49101, 2.84051]
0.44595 0.42042 0.46787 0.44997 [1.62005, 2.08622, 2.49157, 2.84404]
0.44594 0.42076 0.46716 0.44976 [1.62013, 2.08608, 2.49166, 2.84525]
0.44591 0.42131 0.46692 0.44981 [1.62038, 2.08629, 2.49161, 2.84569]
0.44587 0.42109 0.46791 0.44994 [1.62097, 2.08713, 2.49134, 2.84499]
0.44589 0.4199 0.46818 0.4501 [1.62129, 2.08772, 2.49113, 2.84439]
0.44593 0.42131 0.46692 0.44993 [1.62027, 2.08651, 2.49149, 2.84599]
0.44586 0.42014 0.46818 0.45011 [1.6211, 2.08766, 2.49112, 2.84466]
0.44588 0.42109 0.46744 0.4501 [1.62073, 2.0873, 2.49122, 2.84519]
0.44578 0.4199 0.46747 0.45004 [1.62145, 2.08839, 2.49087, 2.84376]
0.44589 0.42109 0.46716 0.44998 [1.62041, 2.08718, 2.49121, 2.84538]
0.4458 0.4199 0.46747 0.45028 [1.62131, 2.08848, 2.49081, 2.84363]
0.44591 0.4199 0.46747 0.45006 [1.62088, 2.08807, 2.49091, 2.84419]
0.44585 0.42061 0.46804 0.45007 [1.62054, 2.08763, 2.49104, 2.84492]
0.44574 0.4208 0.46747 0.44985 [1.62137, 2.08887, 2.49068, 2.84313]
0.44569 0.42082 0.46796 0.44977 [1.62167, 2.08952, 2.49046, 2.84208]
0.44567 0.42154 0.46752 0.44981 [1.62171, 2.08985, 2.49033, 2.84148]
0.44568 0.42202 0.46681 0.44947 [1.62236, 2.09099, 2.48996, 2.83964]
0.44564 0.42273 0.46682 0.44939 [1.62288, 2.09194, 2.48964, 2.83804]
0.4456 0.42349 0.46615 0.4498 [1.62366, 2.09301, 2.4893, 2.83626]
0.44573 0.42383 0.4664 0.45007 [1.62479, 2.09381, 2.48914, 2.83513]
0.44571 0.4236 0.4664 0.45013 [1.62642, 2.0945, 2.48912, 2.83448]
0.44566 0.42377 0.4674 0.45061 [1.62875, 2.09579, 2.4889, 2.83302]
0.44565 0.42312 0.46757 0.45083 [1.63261, 2.09752, 2.48868, 2.83153]
0.44561 0.42318 0.46699 0.4507 [1.63727, 2.0999, 2.48815, 2.82885]
0.4457 0.42119 0.46637 0.45048 [1.64252, 2.10229, 2.4875, 2.82553]
0.44558 0.42243 0.46688 0.45045 [1.6393, 2.10105, 2.48784, 2.82719]
0.44571 0.42096 0.46676 0.45052 [1.6406, 2.10181, 2.48756, 2.8256]
0.44556 0.42266 0.46638 0.45073 [1.63806, 2.10077, 2.48784, 2.82721]
0.44573 0.42084 0.46699 0.45058 [1.63997, 2.10175, 2.48754, 2.82541]
0.44557 0.42197 0.46721 0.45064 [1.6379, 2.10089, 2.48778, 2.82687]
0.44564 0.42106 0.46712 0.45073 [1.63818, 2.10122, 2.48763, 2.82596]
0.44561 0.42129 0.46648 0.45086 [1.63722, 2.1009, 2.48768, 2.82634]
0.44562 0.42129 0.46648 0.45108 [1.63686, 2.1009, 2.48763, 2.82605]
0.44565 0.42081 0.46656 0.45081 [1.63674, 2.10103, 2.48754, 2.82549]
0.44557 0.42115 0.46632 0.45106 [1.63591, 2.10071, 2.48761, 2.8261]
0.4456 0.42104 0.46656 0.45082 [1.63568, 2.10078, 2.48754, 2.82573]
0.44557 0.42104 0.46652 0.45104 [1.6352, 2.10066, 2.48753, 2.82585]
0.44557 0.42104 0.46654 0.4509 [1.63489, 2.10067, 2.48749, 2.82565]
0.44558 0.42104 0.46638 0.45089 [1.63454, 2.10062, 2.48746, 2.82564]
0.44559 0.42169 0.46649 0.45081 [1.6342, 2.10058, 2.48744, 2.82563]
0.44554 0.42169 0.46628 0.45091 [1.63416, 2.10075, 2.48734, 2.82499]
0.44556 0.42111 0.46655 0.45099 [1.63413, 2.10094, 2.48724, 2.82426]
0.44553 0.42097 0.46655 0.45092 [1.6339, 2.10097, 2.48719, 2.82413]
0.44554 0.42119 0.46655 0.45089 [1.63362, 2.10094, 2.48718, 2.8243]
0.44549 0.42097 0.46655 0.45096 [1.6334, 2.10096, 2.48714, 2.82419]
0.4455 0.42128 0.46655 0.45081 [1.63317, 2.10095, 2.48712, 2.82428]
0.44548 0.42106 0.46665 0.45079 [1.63297, 2.10098, 2.48708, 2.82415]
0.44548 0.42128 0.46665 0.45078 [1.63276, 2.10095, 2.48708, 2.82434]
0.44549 0.42128 0.46652 0.45078 [1.63258, 2.10097, 2.48704, 2.82424]
0.44446 0.41904 0.46387 0.44886 [1.72485, 2.13968, 2.4821, 2.86616]
0.44572 0.42273 0.46713 0.45068 [1.64014, 2.10359, 2.48659, 2.83267]
0.44568 0.42297 0.46672 0.45054 [1.6414, 2.10473, 2.48631, 2.83357]
0.44563 0.42343 0.46633 0.45067 [1.6427, 2.10603, 2.48598, 2.83471]
0.44341 0.41926 0.46462 0.44798 [1.4, 2.08459, 2.47808, 2.88939]
0.43846 0.41166 0.4562 0.44147 [1.60875, 1.9, 2.52746, 2.90575]
0.44561 0.42311 0.46646 0.45053 [1.6472, 2.10557, 2.48565, 2.83402]
0.44559 0.4236 0.46568 0.45069 [1.64895, 2.10656, 2.48531, 2.83485]
0.4402 0.41137 0.4663 0.4464 [1.4, 2.21045, 2.48718, 2.8894]
0.44231 0.41627 0.46722 0.44736 [1.4, 2.08154, 2.50315, 2.76363]
0.44555 0.4226 0.46655 0.4504 [1.65139, 2.10705, 2.48499, 2.8382]
0.44553 0.42285 0.46655 0.45046 [1.6518, 2.10746, 2.48483, 2.83838]
0.44553 0.42161 0.46656 0.45053 [1.65251, 2.1081, 2.48458, 2.83882]
0.44556 0.42285 0.46655 0.45052 [1.65178, 2.10757, 2.48476, 2.83818]
0.44552 0.42188 0.46656 0.45051 [1.65243, 2.10816, 2.48453, 2.83859]
0.44556 0.42245 0.46656 0.45048 [1.65204, 2.10792, 2.4846, 2.8382]
0.44554 0.42223 0.46656 0.45039 [1.65217, 2.10815, 2.4845, 2.83822]
0.44554 0.42223 0.46656 0.45051 [1.65202, 2.10813, 2.48449, 2.83803]
0.44205 0.41468 0.46257 0.44738 [1.8, 2.11019, 2.51222, 2.87203]
0.44559 0.42255 0.46594 0.45033 [1.65349, 2.10615, 2.48506, 2.83752]
0.44562 0.42255 0.46605 0.45033 [1.65362, 2.10626, 2.485, 2.83745]
0.43869 0.41219 0.45832 0.44483 [1.78063, 2.21972, 2.46131, 2.86836]
0.44559 0.42282 0.46655 0.45055 [1.65306, 2.10706, 2.48453, 2.83755]
0.44556 0.42228 0.46619 0.45058 [1.65358, 2.10736, 2.48436, 2.83766]
0.44556 0.42228 0.46619 0.45055 [1.65359, 2.10746, 2.4843, 2.83756]
0.44299 0.41893 0.46299 0.44685 [1.49596, 2.08212, 2.49171, 2.97926]
0.44557 0.42249 0.46655 0.45058 [1.65408, 2.10712, 2.4845, 2.83755]
0.44557 0.42249 0.46619 0.45059 [1.65429, 2.10726, 2.48441, 2.8375]
0.4456 0.42228 0.46619 0.45066 [1.65448, 2.10741, 2.48431, 2.83744]
0.43968 0.41324 0.45746 0.44568 [1.75911, 2.11, 2.40617, 2.82586]
0.44556 0.42201 0.46655 0.45077 [1.65453, 2.10738, 2.48455, 2.83787]